In [1]:
# import IJulia
# IJulia.installkernel("Bilevel_DNEP", "--project=$(Base.active_project())")

include("./UpperLevel/UpperLevel.jl")
include("utils.jl")

using ArgParse, PrettyTables
using Plots
using .UpperLevel

In [2]:
include("main.jl")

#--------------------------------------------------------------------------------
#                                                                                
#                                  Bilevel DNEP                                   
#                                                                                
#                                Graduation work                                 
#                                                                                
#--------------------------------------------------------------------------------
# @ Manon Cornet

Running a simulation with the following characteristics:
--------------------------------------------------------

╭─────────────┬───────┬───────┬─────────┬─────────┬─────────┬─────────┬─────────┬───────────╮
│  Model Type │    EV │   EHP │ Delta_t │ PV_CAPA │     EIC │     EEC │   DSOEC │       GCC │
│         [-] │   [-] │   [-] │   [min] │   [MVA] │ [€/kWh] │ [€/kWh] │ [€/kWh] │ [€/kVA/y] │
├─────────

[ Info: Building model...
[ Info: Built model in 6.15 seconds


Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-10
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1
Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1
Set parameter TimeLimit to value 600
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 3479 rows, 2639 columns and 12057 nonzeros
Model fingerprint: 0xd5f13ab6
Model has 144 quadratic constraints
Variable types: 1889 continuous, 750 integer (750 binary)
Coefficient statistics:
  Matrix range     [1e-09, 1e+03]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e-02, 1e+00]
  Bounds range     [9e-01, 1e+00]
  RHS range        [5e-03, 2e+01]
Presolve removed 395 rows and 339 columns
Presolve time: 0.02s
Presolved: 3084 rows, 2300 columns, 10752 nonzeros
Presolved model has 144 quadratic

(68.30584689025758, 7.959727764129639)

In [ ]:
println(network_topology.sending_lines)
println(network_topology.receiving_lines)


In [ ]:
using JuMP
latex_formulation(upper_model)
# Problem is that etiher P_ij or Q_ij_k


In [3]:
objective, time = UpperLevel.solve_model(upper_model, formulation.powerflow)
save_struct(network_topology, "network_topology.json")
save_struct(network, "network_data.json")
# 1. What to do is to update the structures when going through this function
# 2. 


Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 3479 rows, 2639 columns and 12057 nonzeros
Model fingerprint: 0xd5f13ab6
Model has 144 quadratic constraints
Variable types: 1889 continuous, 750 integer (750 binary)
Coefficient statistics:
  Matrix range     [1e-09, 1e+03]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e-02, 1e+00]
  Bounds range     [9e-01, 1e+00]
  RHS range        [5e-03, 2e+01]
Presolved: 3084 rows, 2300 columns, 10752 nonzeros
Presolved model has 144 quadratic constraint(s)

Continuing optimization...


Cutting planes:
  Gomory: 2
  Cover: 7
  Implied bound: 6
  MIR: 23
  Flow cover: 6
  Zero half: 3
  RLT: 4

Explored 1 nodes (4826 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 10 (of 10 available processors)

Solution count 2: 68.3058 69.4996 

Optimal solution found (tolerance 1.00e-02)
Best 

In [78]:
"""Draws a graph using TikZ."""
function showgraph_tikz(network, time_step, x_scale, y_scale;
                        dir=pwd(), filename="graph", display=true)

    preamble = """\\documentclass{standalone}
    \\usepackage{tikz}
    \\usepackage{amsmath}
    \\usetikzlibrary{graphs, quotes, arrows.meta, positioning}
    \\usepackage{xcolor}
    \\definecolor{Ulg_blue}{RGB}{9, 111, 123}
    \\definecolor{Ulg_red}{RGB}{132, 38, 0}
    \\begin{document}
    \\begin{tikzpicture}[
        every label/.style = {align=center, font=\\tiny, inner sep=2pt},
        every edge quotes/.style = {font=\\scriptsize, text=black, fill=white, inner sep=2pt}
        ]
    \\graph [no placement]
    {\n"""

    postamble = """};
    \\end{tikzpicture}
    \\end{document}"""
 
    style_sub = "rectangle, draw, fill=white, minimum size=1em, inner sep=1pt,  align=center, text width=0.6cm"
    style_load = "circle, draw, fill=white, minimum size=1em, inner sep=1pt, align=center, text width=0.6cm"
    base_power = network.pu_basis.base_power

    # Creating the .tex file
    file_path = joinpath(dir ,"$filename.tex")
    touch(file_path)

    # Writing the .tex file
    open(file_path, "w") do file
        write(file, preamble)
        
        for b in network.buses
            node = b.node
            n = node.id
            x_coord = node.coord.x
            y_coord = node.coord.y
            if n in [1, 2]
                x_coord += 0.5
            elseif n == 16
                x_coord += 0.1
                y_coord += 0.1
            elseif n == 20
                y_coord -=  0.1
            elseif n in [3, 9]
                if n == 9 
                    x_coord += 0.5  
                end
                y_coord -= 1.8
            elseif n in [4, 5]
                if n == 4
                    x_coord += 0.8
                else 
                    x_coord += 0.5
                end
                y_coord += 0.4
            end
            x = x_scale * x_coord
            y = y_scale * y_coord
            
            ns = get_nb_substations(network)
            nu = n - ns
            if n <= ns
                P_gen = round((b.P_sup[time_step] * base_power); digits = 2)
                col = P_gen >= 0 ? "\\color{Ulg_blue}" : "\\color{Ulg_red}"
                if b.built
                    write(file,
                    "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{S}_{$n}\$  \\vspace{0.1em}  \\scriptsize \$\\mathbf{$col $P_gen}\$}, $(style_sub), ultrathick];\n")
                else 
                    write(file,
                    "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{S}_{$n}\$ \\vspace{0.1em}  \\scriptsize \$\\mathbf{$col $P_gen}\$}, $(style_sub)];\n")
                end
            else 
                P_cons = round(b.load_profile.time_serie[time_step] * base_power; digits = 2)
                P_gen = isnothing(b.PV_installation) ? 0.0 : b.PV_installation.P[time_step]*base_power 
                P_gen = round(P_gen; digits=2)
                write(file,
                "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{U}_{$nu}\$  \\vspace{-0.2em} \\scriptsize  \$ \\color{Ulg_blue} \\mathbf{$P_gen}\$ \\vspace{0.1em} \\scriptsize \$ \\color{Ulg_red} \\mathbf{$P_cons}\$}, $(style_load)];\n")
            end
        end

        for l in network.lines
            if l.built
                if l.P_send[time_step] >= 0
                    i = l.edge.from_node.id ; j = l.edge.to_node.id
                    p = l.P_send[time_step] * base_power
                else 
                    i = l.edge.to_node.id ; j = l.edge.from_node.id
                    p = abs(l.P_rec[time_step]) * base_power
                end
                write(file,
                "    $(i) ->[thick, \"$(round(p, digits=2))\"] $(j);\n")
            else 
                i = l.edge.from_node.id ; j = l.edge.to_node.id
                write(file,
                "    $(i) --[gray, dashed] $(j);\n")
            end       
        end  
        write(file, postamble)
    end

    if display
        run(Cmd(`lualatex $filename.tex`, dir="$dir"))
        run(Cmd(`open $filename.pdf`, dir="$dir"))
    end
end

showgraph_tikz

In [ ]:
"""Draws a graph using TikZ."""
function showgraph_tikz(network, time_step, x_scale, y_scale;
                        dir=pwd(), filename="graph", display=true)

    preamble = """\\documentclass{standalone}
    \\usepackage{tikz}
    \\usepackage{amsmath}
    \\usetikzlibrary{graphs, quotes, arrows.meta, positioning}

    \\begin{document}
    \\begin{tikzpicture}[
        every label/.style = {align=center, font=\\tiny, inner sep=2pt},
        every edge quotes/.style = {font=\\scriptsize, text=black, fill=white, inner sep=2pt}
        ]
    \\graph [no placement]
    {\n"""

    postamble = """};
    \\end{tikzpicture}
    \\end{document}"""
 
    style_sub = "rectangle, draw, fill=white, minimum size=1em, inner sep=1pt"
    style_load = "circle, draw, fill=white, minimum size=1em, inner sep=1pt, align=center, text width=0.5cm"
    base_power = network.pu_basis.base_power

    # Creating the .tex file
    file_path = joinpath(dir ,"$filename.tex")
    touch(file_path)

    # Writing the .tex file
    open(file_path, "w") do file
        write(file, preamble)
        
        for b in network.buses
            node = b.node
            n = node.id
            x = x_scale * node.coord.x
            y = y_scale * node.coord.y
            ns = get_nb_substations(network)
            nu = n - ns
            if n <= ns
                P_gen = round((b.P_sup[time_step] * base_power); sigdigits = 3)
                if b.built
                    write(file,
                    "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{S}_{$n}\$  \\tiny  \$ $P_gen\$}, $(style_sub), thick];\n")
                else 
                    write(file,
                    "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{S}_{$n}\$ \\tiny \$ $P_gen\$}, $(style_sub)];\n")
                end
            else 
                P_cons = round(b.load_profile.time_serie[time_step] * base_power; sigdigits = 3)
                P_gen = isnothing(b.PV_installation) ? 0.0 : b.PV_installation.P[time_step]*base_power 
                P_gen = round(P_gen; sigdigits=3)
                write(file,
                "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{U}_{$nu}\$  \\newline \\tiny  \$+$P_gen-$P_cons\$}, $(style_load)];\n")
            end
        end

        for l in network.lines
            if l.built
                if l.P_send[time_step] >= 0
                    i = l.edge.from_node.id ; j = l.edge.to_node.id
                    p = l.P_send[time_step] * base_power
                else 
                    i = l.edge.to_node.id ; j = l.edge.from_node.id
                    p = abs(l.P_rec[time_step]) * base_power
                end
                write(file,
                "    $(i) ->[thick, \"$(round(p, sigdigits=3))\"] $(j);\n")
            else 
                i = l.edge.from_node.id ; j = l.edge.to_node.id
                write(file,
                "    $(i) --[gray, dashed] $(j);\n")
            end       
        end  
        write(file, postamble)
    end

    if display
        run(Cmd(`lualatex $filename.tex`, dir="$dir"))
        run(Cmd(`open $filename.pdf`, dir="$dir"))
    end
end

In [ ]:
# --- https://tikz.dev/tikz-graphs#sec-19.7 ---
"""Draws a graph using TikZ."""
function showgraph_tikz(network, time_step, x_scale, y_scale;
                   dir=pwd(),
                   filename="graph",
                   display=true)

    # Let's insert some boilerplate styling
    # and necessary preamble/postamble
    preamble = """\\documentclass{standalone}
    \\usepackage{tikz}
    \\usepackage{amsmath}
    \\usetikzlibrary{graphs, quotes, arrows.meta, positioning}

    \\begin{document}
    \\begin{tikzpicture}[
        every label/.style = {align=center, font=\\tiny, inner sep=2pt},
        every edge quotes/.style = {font=\\scriptsize, text=black, fill=white, inner sep=2pt}
        ]
    \\graph [no placement]
    {\n"""

    
    postamble = """};
    \\end{tikzpicture}
    \\end{document}"""
 
    style_sub = "rectangle, draw, fill=white, minimum size=1em, inner sep=1pt"
    style_load = "circle, draw, fill=white, minimum size=1em, inner sep=1pt"
    base_power = network.pu_basis.base_power

    file_path = joinpath(dir ,"$filename.tex")

    touch(file_path)

    open(file_path, "w") do file
        write(file, preamble)
        
        for b in network.buses
            node = b.node
            n = node.id
            x = x_scale * node.coord.x
            y = y_scale * node.coord.y
            if n <= get_nb_substations(network)
                write(file,
                "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{S}_{$n}\$}, $(style_sub)];\n")
            else 
                write(file,
                "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{L}_{$n}\$}, $(style_load)];\n")
            end
        end

        for l in network.lines
            if l.built
                if l.P_send[time_step] >= 0
                    i = l.edge.from_node.id ; j = l.edge.to_node.id
                    p = l.P_send[time_step] * base_power
                else 
                    i = l.edge.to_node.id ; j = l.edge.from_node.id
                    p = abs(l.P_rec[time_step]) * base_power
                end
                write(file,
                "    $(i) ->[thick, \"$(round(p, digits=3))\"] $(j);\n")
            else 
                i = l.edge.from_node.id ; j = l.edge.to_node.id
                write(file,
                "    $(i) --[gray, dashed] $(j);\n")
            end       
        end  
        write(file, postamble)
    end

    # command = `cd $dir` |> `lualatex $filename.tex` |> `open $filename.pdf`
    if display
        run(Cmd(`lualatex $filename.tex`, dir="$dir"))
        run(Cmd(`open $filename.pdf`, dir="$dir"))
        # run(`cd $dir` |> `lualatex $filename.tex` |> `open $filename.pdf`)
    end
end

In [79]:
# -- Creation of the result plot --
using Graphs

g = Graph(get_nb_nodes(network_topology))
for l in network.lines 
    e = l.edge
    add_edge!(g, e.from_node.id, e.to_node.id)
end

root_dir = splitdir(@__DIR__)[1]
plots_dir = joinpath(root_dir, "plots")
thesis_dir = "/Users/manoncornet/Documents/University/TFE/ThesisWriting/Master_Thesis/figures" 

for t in 1:simulation.nb_time_steps
    showgraph_tikz(network, t, 5.5, 4.5; dir=thesis_dir, filename="graph_$(t)")
end

# file_path = joinpath(plots_dir ,"graph1.tex")
# touch(file_path)
# run(`cd $plots_dir open graph1.tex`) 

This is LuaHBTeX, Version 1.15.0 (TeX Live 2022) 
 restricted system commands enabled.
(./graph_1.tex
LaTeX2e <2021-11-15> patch level 1
 L3 programming layer <2022-02-24>
(/usr/local/texlive/2022/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2018/03/26 v1.3a Class to compile TeX sub-files stan
dalone
(/usr/local/texlive/2022/texmf-dist/tex/latex/tools/shellesc.sty)
(/usr/local/texlive/2022/texmf-dist/tex/generic/iftex/ifluatex.sty
(/usr/local/texlive/2022/texmf-dist/tex/generic/iftex/iftex.sty))
(/usr/local/texlive/2022/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/xkeyval.tex
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/xkvutils.tex
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/keyval.tex))))
(/usr/local/texlive/2022/texmf-dist/tex/latex/standalone/standalone.cfg)
(/usr/local/texlive/2022/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX docu

In [ ]:
function communicate(cmd::Cmd, input)
    inp = Pipe()
    out = Pipe()
    err = Pipe()

    process = run(pipeline(cmd, stdin=inp, stdout=out, stderr=err), wait=false)
    close(out.in)
    close(err.in)

    stdout = @async String(read(out))
    stderr = @async String(read(err))
    write(process, input)
    close(inp)
    wait(process)
    return (
        stdout = fetch(stdout),
        stderr = fetch(stderr),
        code = process.exitcode
    )
end


a, = communicate(`julia --project ./main.jl`, "output.txt"); println(a)


In [ ]:
using Graphs
using GraphViz
G = Graph(3) # graph with 3 vertices

# make a triangle
add_edge!(G, 1, 2)
add_edge!(G, 1, 3)
add_edge!(G, 2, 3)

for e in edges(G)
    print(e)
end
# print(nv(G))
# W=zeros(Float64, nv(G), nv(G))
# print(W[1,1])
# for i in 1:nv(G)
#     for j in 1:nv(G)
#         if has_edge(G, i, j)
#             print(i,j)
#             W[i, j] = 0.1
#         end
#     end
# end 


# open("./graph.tmp.dot", "w") do file
#     write(file, "digraph {\n")

#     for n in vertices(G)  # add nodes
#         write(file,
#               "    $n;\n")
#     end

#     for e in edges(G)  # add edges
#         i=src(e); j=dst(e)
#         write(file,
#               "    $i -> $j;\n")
#     end

#     write(file, "}")
# end

# GraphViz.load("graph.tmp.dot")
# current_path = pwd()
# print(current_path)

# import Base
# Base.run(`dot -V`)
# Base.run(`cd $current_path`)
# Base.run(pipeline(`dot -Tpdf graph.tmp.dot`,
#                stdout="graph.tmp.pdf"))
# Base.run(`open graph.tmp.pdf`)


In [ ]:
import Base
Base.run(`dot -V`)

In [ ]:
import Base

Base.run(`ls`)

In [ ]:
# Represent graphically the model 
# the use of graphviz is better !!!
import Pkg; Pkg.add("GraphViz")
using GraphViz
# I really like graphviz, try with this library
open("./graph.tmp.dot", "w") do file
    write(file, "digraph {\n")

    for n in vertices(g)  # add nodes
        write(file,
              "    $n;\n")
    end

    for e in edges(g)  # add edges
        i=src(e); j=dst(e)
        write(file,
              "    $i -> $j;\n")
    end

    write(file, "}")
end
run(pipeline(`dot -Tpdf ./graph.tmp.dot`,
             stdout="./graph.tmp.pdf"))
run(`xdg-open $filename.pdf`, wait=false)

dot"""
graph graphname {
     // The label attribute can be used to change the label of a node
     a [label="Sub"];
     // Here, the node shape is changed.
     b [shape=house][color=grey];
     // These edges both have different line properties
     a -- b -- c [color=grey];
     b -- d [style=dotted];
 }
"""